In [ ]:
# =======================
# 0) Constants & helpers
# =======================
using Serialization, CSV, DataFrames, SparseArrays, Distributions, StatsBase

const GENOME_LEN = 367_808
const N = GENOME_LEN

# Significance thresholds
const THRESH_TvsC_SIG = 0.75
const THRESH_TvsU_SIG = 0.95
const THRESH_DEFAULT   = 0.95
const THRESH_CvsU_SIG  = 0.80

# Reverse-complement index helpers (1-based)
rc(x::Int) = N - x + 1
rc(r::UnitRange{Int}) = (N - last(r) + 1):(N - first(r) + 1)

# If you want reverse positions as -(N - i) instead of -(N - i + 1), set true.
const RC_MINUS_ONE = false  # false => genomic pos = -(N - i + 1); true => -(N - i)

# =======================
# 1) Load GFF & masking
# =======================
gff = CSV.File("/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/RF libraries/BK010421.gff3";
               comment="#",
               header=["accession","software","feature","start","stop","score","strand","phase","attributes"]) |> DataFrame
tomask = filter(x -> x.feature ∈ ["rRNA","tRNA"], gff)

function mask!(data::AbstractVector{<:Real})
    for f in eachrow(tomask)
        data[f.start:f.stop] .= 0
    end
    return data
end

# =======================
# 2) Mapping struct
# =======================
struct Mapping
    readrange::UnitRange{Int32}   # genomic (plus-orientation)
    strand::Char                  # '+' or '-'
    strandrange::UnitRange{Int32} # as provided by mapper (may be RC)
end

# ==========================================
# 3) Termini container + I/O
# ==========================================
struct Termini{T}
    fwd5::Vector{T}
    rev5::Vector{T}
    fwd3::Vector{T}
    rev3::Vector{T}
end

# Original semantics: place raw counts into forward/reverse arrays by strand.
function readcounts(filepath::String)
    counts = Termini{Int}(spzeros(Int, GENOME_LEN), spzeros(Int, GENOME_LEN),
                          spzeros(Int, GENOME_LEN), spzeros(Int, GENOME_LEN))
    for (m1, m2) in deserialize(filepath)
        if m1.strand == '+'
            counts.fwd3[mod1(last(m1.strandrange),  GENOME_LEN)] += 1
            counts.fwd5[mod1(first(m2.strandrange), GENOME_LEN)] += 1
        else
            counts.rev3[mod1(last(m1.strandrange),  GENOME_LEN)] += 1
            counts.rev5[mod1(first(m2.strandrange), GENOME_LEN)] += 1
        end
    end
    return counts
end

# Sum counts across a group of files
function sum_group_counts(paths::Vector{String})
    acc = Termini{Int}(zeros(Int, GENOME_LEN), zeros(Int, GENOME_LEN),
                       zeros(Int, GENOME_LEN), zeros(Int, GENOME_LEN))
    for p in paths
        c = readcounts(p)
        @inbounds begin
            acc.fwd5 .+= c.fwd5
            acc.rev5 .+= c.rev5
            acc.fwd3 .+= c.fwd3
            acc.rev3 .+= c.rev3
        end
    end
    return acc
end

# =======================
# 4) Define matched trios (no replicates; per-sample analysis)
# =======================
struct Trio
    name::String   # condition label
    C::String      # ligated
    U::String      # unligated
    T::String      # TAP-ligated
end

const TRIOS = [
    Trio("mRPF2-atp1",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/C1.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/U6.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/T1.mappings3.bin"),

    Trio("mRPF2-nad6",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/C3.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/U8.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/T3.mappings3.bin"),

    Trio("RPF2",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/C4.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/U9.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/T4.mappings3.bin"),

    Trio("Col-0",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/C5.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/U10.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/T5.mappings3.bin"),

    Trio("rpf2",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/C12.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/U11.mappings3.bin",
         "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3/T6.mappings3.bin"),
]

# =======================
# 5) Per-trio: load counts (no pooling) & mask
# =======================
function load_and_mask(filepath::String)
    c = readcounts(filepath)
    mask!.(Ref(c.fwd5)); mask!.(Ref(c.rev5)); mask!.(Ref(c.fwd3)); mask!.(Ref(c.rev3))
    return c
end

# =======================
# 6) Convert to ±pos DataFrames (helper kept identical)
# =======================
@inline function neg_genomic_pos(i::Int)
    if RC_MINUS_ONE
        return -(N - i)
    else
        return -rc(i)  # -(N - i + 1)
    end
end

function termini_to_df(name::String, fwd::AbstractVector{<:Real}, rev::AbstractVector{<:Real})
    df = DataFrame(:pos => Int[], Symbol(name) => Float64[])
    @inbounds for (i, v) in enumerate(fwd)
        push!(df, (i, float(v)))
    end
    @inbounds for (i, v) in enumerate(rev)
        push!(df, (neg_genomic_pos(i), float(v)))
    end
    return df
end

betamean(c1, c2) = mean(Beta(c1 + 1, c2 + 1))

# =======================
# 7) Per-trio analysis and exports
# =======================
outroot = "/Users/lenic/Library/CloudStorage/OneDrive-UWA/PhD/Research/01 Transcript End Analysis/Bioinformatics/2025/RF/mappings3"
subdir  = "beta_distributions/beta_normalised/noIRfold"
outdir  = joinpath(outroot, subdir)
isdir(outdir) || mkpath(outdir)
savecsv(fname::AbstractString, df) = CSV.write(joinpath(outdir, fname), df)

combined_all = DataFrame()  # optional aggregator

for trio in TRIOS
    # Load
    Cc = load_and_mask(trio.C)
    Uc = load_and_mask(trio.U)
    Tc = load_and_mask(trio.T)

    # 5′ and 3′ per sample
    C5 = termini_to_df("ligated5",   Cc.fwd5, Cc.rev5)
    U5 = termini_to_df("unligated5", Uc.fwd5, Uc.rev5)
    T5 = termini_to_df("tap5",       Tc.fwd5, Tc.rev5)

    C3 = termini_to_df("ligated3",   Cc.fwd3, Cc.rev3)
    U3 = termini_to_df("unligated3", Uc.fwd3, Uc.rev3)
    T3 = termini_to_df("tap3",       Tc.fwd3, Tc.rev3)

    # Join within trio
    fiveprime  = innerjoin(innerjoin(U5, C5; on=:pos), T5; on=:pos)
    threeprime = innerjoin(innerjoin(U3, C3; on=:pos), T3; on=:pos)

    # Trio-specific scaling (shared positions above count threshold)
    shared5 = filter(x -> minimum((x.unligated5, x.ligated5, x.tap5)) > 9, fiveprime)
    geomeans5 = [geomean((p.unligated5, p.ligated5, p.tap5)) for p in eachrow(shared5)]
    unligated5_sf = median(shared5.unligated5 ./ geomeans5)
    ligated5_sf   = median(shared5.ligated5   ./ geomeans5)
    tap5_sf       = median(shared5.tap5       ./ geomeans5)

    shared3 = filter(x -> minimum((x.unligated3, x.ligated3, x.tap3)) > 9, threeprime)
    geomeans3 = [geomean((p.unligated3, p.ligated3, p.tap3)) for p in eachrow(shared3)]
    unligated3_sf = median(shared3.unligated3 ./ geomeans3)
    ligated3_sf   = median(shared3.ligated3   ./ geomeans3)
    tap3_sf       = median(shared3.tap3       ./ geomeans3)

    # Beta means within trio
    fiveprime.CvsU = [betamean(r.ligated5/ligated5_sf, r.unligated5/unligated5_sf) for r in eachrow(fiveprime)]
    fiveprime.TvsC = [betamean(r.tap5/tap5_sf,         r.ligated5/ligated5_sf)     for r in eachrow(fiveprime)]
    fiveprime.TvsU = [betamean(r.tap5/tap5_sf,         r.unligated5/unligated5_sf) for r in eachrow(fiveprime)]

    threeprime.CvsU = [betamean(r.ligated3/ligated3_sf, r.unligated3/unligated3_sf) for r in eachrow(threeprime)]
    threeprime.TvsC = [betamean(r.tap3/tap3_sf,         r.ligated3/ligated3_sf)     for r in eachrow(threeprime)]
    threeprime.TvsU = [betamean(r.tap3/tap3_sf,         r.unligated3/unligated3_sf) for r in eachrow(threeprime)]

    # Tag with trio name for downstream grouping
    insertcols!(fiveprime, 1, :sample => fill(trio.name, nrow(fiveprime)))
    insertcols!(threeprime,1, :sample => fill(trio.name, nrow(threeprime)))

    # === Exports per trio ===
    base = replace(trio.name, r"\s" => "_")
    savecsv("$(base)_beta_norm_CvsU_all_5p.csv", filter(:CvsU => x -> x > 0, fiveprime))
    savecsv("$(base)_beta_norm_CvsU_all_3p.csv", filter(:CvsU => x -> x > 0, threeprime))
    savecsv("$(base)_beta_norm_TvsC_all_5p.csv", filter(:TvsC => x -> x > 0, fiveprime))
    savecsv("$(base)_beta_norm_TvsC_all_3p.csv", filter(:TvsC => x -> x > 0, threeprime))
    savecsv("$(base)_beta_norm_TvsU_all_5p.csv", filter(:TvsU => x -> x > 0, fiveprime))
    savecsv("$(base)_beta_norm_TvsU_all_3p.csv", filter(:TvsU => x -> x > 0, threeprime))

    savecsv("$(base)_beta_norm_CvsU_sig095_5p.csv", filter(:CvsU => x -> x >= THRESH_DEFAULT, fiveprime))
    savecsv("$(base)_beta_norm_CvsU_sig095_3p.csv", filter(:CvsU => x -> x >= THRESH_DEFAULT, threeprime))
    savecsv("$(base)_beta_norm_CvsU_sig080_5p.csv", filter(:CvsU => x -> x >= THRESH_CvsU_SIG, fiveprime))
    savecsv("$(base)_beta_norm_CvsU_sig080_3p.csv", filter(:CvsU => x -> x >= THRESH_CvsU_SIG, threeprime))

    savecsv("$(base)_beta_norm_TvsC_sig095_5p.csv", filter(:TvsC => x -> x >= THRESH_DEFAULT, fiveprime))
    savecsv("$(base)_beta_norm_TvsC_sig095_3p.csv", filter(:TvsC => x -> x >= THRESH_DEFAULT, threeprime))
    savecsv("$(base)_beta_norm_TvsC_sig075_5p.csv", filter(:TvsC => x -> x >= THRESH_TvsC_SIG, fiveprime))
    savecsv("$(base)_beta_norm_TvsC_sig075_3p.csv", filter(:TvsC => x -> x >= THRESH_TvsC_SIG, threeprime))

    savecsv("$(base)_beta_norm_TvsU_sig095_5p.csv", filter(:TvsU => x -> x >= THRESH_TvsU_SIG, fiveprime))
    savecsv("$(base)_beta_norm_TvsU_sig095_3p.csv", filter(:TvsU => x -> x >= THRESH_TvsU_SIG, threeprime))

    # Optional combined output for "both significant 5′" per trio
    sig_both_5 = filter(r -> (r.TvsC >= THRESH_TvsC_SIG) && (r.TvsU >= THRESH_TvsU_SIG), fiveprime)
    if nrow(sig_both_5) > 0
        out = deepcopy(sig_both_5)
        insertcols!(out, 2, :end => fill("5p", nrow(out)))
        savecsv("$(base)_beta_norm_TvsC075_TvsU095_bothsig_5p.csv", out)
    end
end
